# Best Model, train for 100 epoch, NAdam, larger dataset.


In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from VGGs import *
from MobileNetV3 import *
from engine import *
from efficientNet import *

change_to_disk()
data_dir = Path("clean_split_5k/")
models_path = Path('Models/')
saves = Path('Trainning_results_saves/')

In [2]:
data_transform = transforms.Compose([
    # Resize our images to 224x224
    transforms.Resize(size=(224, 224)),
    
    # Rotate the images randomly (just to make it as independent from the position of the pipeline)
    transforms.RandomRotation(180),

    # Random crop
    #transforms.RandomCrop(140),

    #Random horizontal flip
    transforms.RandomHorizontalFlip(p=0.5),

    # Turn the image into a torch.Tensor
    transforms.ToTensor(),

    # Normalize
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

BATCH_SIZE = 64
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size = BATCH_SIZE)

In [3]:
# Things to play with
# Hyperparameters -> NUM_EPOCHS - 100 | Learning_rate - 0.001 
# Functions -> Optimizer - Rprop | Loss Function - Cross Entropy Loss

NUM_EPOCHS = 150
lr = 0.001
device = get_device()

mobnet_sml = MobileNetV3("small")

loss_fn = nn.CrossEntropyLoss()
nadam_optim = torch.optim.NAdam(params=mobnet_sml.parameters(), lr=lr)

In [5]:
mobnet_sml.to(device)
save_name = "mobnet_sml"
train_mobnet_sml_results, train_time_mobnet_sml = train(mobnet_sml, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=save_name, device=device)

  0%|          | 0/150 [00:00<?, ?it/s]

Saving the model to: Models\mobnet_sml_0_epc.pth
Epoch: 1 | train_loss: 0.4587 | train_acc: 0.7913 | validation_loss: 0.6330 | validation_acc: 0.7828
Saving the model to: Models\mobnet_sml_1_epc.pth
Epoch: 2 | train_loss: 0.2894 | train_acc: 0.8785 | validation_loss: 0.4361 | validation_acc: 0.8520
Saving the model to: Models\mobnet_sml_2_epc.pth
Epoch: 3 | train_loss: 0.2309 | train_acc: 0.9078 | validation_loss: 0.3614 | validation_acc: 0.8914
Saving the model to: Models\mobnet_sml_3_epc.pth
Epoch: 4 | train_loss: 0.2065 | train_acc: 0.9171 | validation_loss: 0.3161 | validation_acc: 0.9098
Epoch: 5 | train_loss: 0.1723 | train_acc: 0.9318 | validation_loss: 0.5836 | validation_acc: 0.8484
Epoch: 6 | train_loss: 0.1601 | train_acc: 0.9348 | validation_loss: 0.9383 | validation_acc: 0.7809
Epoch: 7 | train_loss: 0.1622 | train_acc: 0.9366 | validation_loss: 0.7965 | validation_acc: 0.7891
Epoch: 8 | train_loss: 0.1495 | train_acc: 0.9409 | validation_loss: 0.4106 | validation_acc: 0.9

KeyboardInterrupt: 

In [9]:
model_path = Path("Models/mobnet_sml_2_epc.pth")
loaded_model = MobileNetV3("small")
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.to(device)

MobileNetV3(
  (conv): ConvBlock(
    (c): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): Hardswish()
  )
  (blocks): ModuleList(
    (0): BNeck(
      (block): Sequential(
        (0): ConvBlock(
          (c): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU()
        )
        (1): ConvBlock(
          (c): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU()
        )
        (2): SeBlock(
          (globpool): AdaptiveAvgPool2d(output_size=(1, 1))
          (fc1): Linear(in_features=16, out_features=4, bias=False)
          (fc2): Linear(in_features=4, out_features=16, b

In [10]:
eval_mobnet_sml_results = eval_model(loaded_model, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))

  0%|          | 0/40 [00:27<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 11.
Original Traceback (most recent call last):
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 143, in <listcomp>
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  #

In [ ]:
eval_mobnet_sml_results

{'model_name': 'MobileNetV3',
 'model_loss': 368.15277099609375,
 'model_acc': 21.640625,
 'model_inf_time': 41.67718324661255}

In [ ]:
plot_loss_curves(train_mobnet_sml_results,"MobileNet V3 Small")

In [ ]:
plot_confusion_matrix(mobnet_sml, test_dl, device, class_names, "MobileNetV3 Small")